# Notebook: View Dataset Statistics

This notebook is used to explore a given dataset.

## Packages

In [1]:
import pandas as pd
import tiktoken
import re

## Constants

In [2]:
DATASET_PATH = "reviews_dataset/balanced_reviews_sentences.csv"
LLM_MODEL_NAME = "gpt-3.5-turbo"

## Code

### General Statistics

In [3]:
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0.1,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,detected_language
0,0,1537,867609212,743219,1,Schönes Ambiente,2022-11-07,schneiderrebecca,"Würzburg, Deutschland",Das Personal freundlich.,4.0,leipzig,Cafe Riquet,de,Ich war gegen 17:30 Uhr in dem Kaffeehaus. Die...,4,de
1,1,2443,911273338,1342313,1,Nicht nur überaus köstlich in romantischer Atm...,2023-08-18,Michael E,NaN,"Überragender Service, köstliche Speisen!Wir ko...",5.0,bielefeld,Jivino Enoteca,de,"Ungefragt wurde uns vorab serviert, was wir mö...",1,de
2,2,2150,910109779,1118853,0,Einfache Biergartenküche zu gehobenen Preisen,2023-08-13,ingrid240,"Frankfurt am Main, Deutschland","Freundlicher Service, sehr gutes Bier.Das <LOC...",2.0,hannover,Bavarium,de,"Freundlicher Service, sehr gutes Bier.Das Tiro...",0,de
3,3,724,857521547,1190751,12,Schlemmermenü super köstlich.,2022-08-31,409gisell,NaN,Jeder Gang ein Genuss.,5.0,frankfurt,Trattoria i Siciliani,de,Jeder Gang ein Genuss. Wir haben Fisch und Fle...,0,de
4,4,2984,892846168,2481134,1,"Essen, Wein und Service sehr zu empfehlen",2023-05-31,Explore04799982138,"Gaiole in Chianti, Italien",Ein tolles italienisches Restaurant.,4.0,mannheim,Da Vino,de,Ein tolles italienisches Restaurant. Essen tol...,0,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3495,504,868718065,7235024,2,Mein erster originaler rheinischer Sauerbraten,2022-11-16,ALMSUPA,"Wien, Österreich",Danke für die tolle Zeit bei Euch!,4.0,köln,Früh am Dom,de,"Waren in der City mit dem ""einheimischen"" Cous...",5,de
3496,3496,218,883034414,954454,2,War mit 4 Freunden hier essen. Freundliches Pe...,2023-03-21,Joker-BN,"Bonn, Deutschland",Nach Rücksprache mit dem Chef wurde eine Ausna...,4.0,hamburg,ALEX Hamburg,de,Sehr schöne Lage mit Blick auf die Alster. Gut...,14,de
3497,3497,2205,853942330,1390048,2,"Gutes Essen, meist mit Wartezeit",2022-08-13,TobiMTravel,"München, Deutschland",Die <LOC> waren sehr gut und das Sauerkraut eb...,4.0,nürnberg,Bratwursthäusle,de,Wir waren als paar an einem Samstag Mittag dor...,1,de
3498,3498,3175,889498235,7322855,4,Ein Reines Gedicht,2023-05-08,Tim R,NaN,Es schmeckt als ob die Kuh jeden Tag vom Bauer...,5.0,augsburg,SteakManufaktur,de,Zur Ambiente:Wir würden herzlich empfangen vom...,11,de


In [4]:
rating_counts = df['rating'].value_counts()
rating_counts

rating
4.0    700
5.0    700
2.0    700
3.0    700
1.0    700
Name: count, dtype: int64

In [5]:
rating_counts = df['city'].value_counts()
rating_counts

city
hamburg            375
leipzig            282
dresden            259
münchen            231
düsseldorf         171
frankfurt          161
bonn               156
stuttgart          156
köln               155
nürnberg           140
münster            126
berlin             121
augsburg           121
mannheim           118
bremen             114
bielefeld          108
mönchengladbach    105
bochum             101
hannover            97
essen               96
wiesbaden           94
karlsruhe           81
dortmund            55
wuppertal           45
duisburg            32
Name: count, dtype: int64

In [6]:
rating_counts = df['restaurant_name'].value_counts()
rating_counts

restaurant_name
Bullerei                    173
Ratskeller                  148
Mamma Mia                   142
Olive                        89
Schneider Bräuhaus           84
                           ... 
Ola Lisboa                    3
Argentina - Steak & Mehr      3
Tokyo Sushi Bar               2
Thai Banyan Restaurant        2
Webster-Brauhaus              1
Name: count, Length: 122, dtype: int64

In [7]:
#df.groupby(['sentence_idx', 'review_id']).size().reset_index(name='count')

In [8]:
def count_words(text):
    words = re.findall(r'\w+', text)
    return len(words)

df['word_count'] = df['text'].apply(count_words)
average_word_count_by_rating = df.groupby('rating')['word_count'].mean()
average_word_count_by_rating

rating
1.0    15.238571
2.0    14.152857
3.0    14.197143
4.0    12.604286
5.0    11.487143
Name: word_count, dtype: float64

In [9]:
 df['word_count'].mean()

13.536

### Count Tokens (for LLM)

In [10]:
# Source: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_string(string: str, model_name: str) -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [14]:
df['llm_token_count'] = df['text'].apply(lambda x: num_tokens_from_string(x, LLM_MODEL_NAME))

In [19]:
pd.set_option('display.max_colwidth', None)

In [38]:
max_token_row = df[df['llm_token_count'] == df['llm_token_count'].max()]
print("Zeile mit den meisten Tokens:")
max_token_row#["llm_token_count"].values[0]

Zeile mit den meisten Tokens:


,Unnamed: 0.1,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,detected_language,word_count,llm_token_count
2856,2856,3122,887348853,908561,0,Willkommen in der Servicewüste „Anders am Turmberg“!,2023-04-23,Michael R,"Landau in der Pfalz, Deutschland","Der Chef steht mit Straßenkleidung in der Küche und lacht nur, über was war nicht klar – kein Wort des Bedauerns.Der Aperitif wurde nachgereicht und als Kompensation für ruiniertes Kleid und Handtasche zugegebenermaßen nicht berechnet.Schlechter Service scheint aber zum Ton des Hauses zu gehören, denn das Essen wurde serviert und erst danach erhielt man das Besteck direkt auf die Hand, Servietten lagen lieblos auf dem Tisch verstreut bzw. musste man anfordern und dafür zahlt man dann Spitzenpreise!Schaut man sich die Rezensionen über diese Lokalität auf den verschiedenen Foren an, so lebt die Kneipe fast nur von der Lage mit dem guten Blick auf Karlsruhe, aber dafür muss man Gottseidank nicht in das Lokal gehen!“ Im Übrigen hat auch uns fast jedes der dort monierten „Kleinigkeiten“ betroffen.Zusammengefasst ist dieses Etablissement nicht empfehlenswert und für den gebotenen Service zu teuer – Karlsruhe hat viele, deutlich bessere Lokale und unter gehobener Gastronomie stelle ich mir etwas anderes vor.",1.0,karlsruhe,Anders auf dem Turmberg,de,"2 volle Gläser mit Aperitif von einem Azubi über meine Frau geschüttet. So etwas kann ja mal passieren, auch wenn es in durchnässter Kleidung nicht mehr so schmeckt und zudem die Handtasche und der Sessel betroffen sind. Schlimm ist nur, dass weder eine richtige Entschuldigung kam, der Stuhl nicht gewechselt wurde und der Boden nicht gewischt wurde. Auch die Scherben wurden nicht alle entfernt. Hilfe oder Unterstützung, das Missgeschick zu beseitigen – Fehlanzeige.Lediglich der Hinweis, wir klären das später – toll, wenn einem keiner hilft, den Schaden zu begrenzen. Der Chef steht mit Straßenkleidung in der Küche und lacht nur, über was war nicht klar – kein Wort des Bedauerns.Der Aperitif wurde nachgereicht und als Kompensation für ruiniertes Kleid und Handtasche zugegebenermaßen nicht berechnet.Schlechter Service scheint aber zum Ton des Hauses zu gehören, denn das Essen wurde serviert und erst danach erhielt man das Besteck direkt auf die Hand, Servietten lagen lieblos auf dem Tisch verstreut bzw. musste man anfordern und dafür zahlt man dann Spitzenpreise!Schaut man sich die Rezensionen über diese Lokalität auf den verschiedenen Foren an, so lebt die Kneipe fast nur von der Lage mit dem guten Blick auf Karlsruhe, aber dafür muss man Gottseidank nicht in das Lokal gehen!“ Im Übrigen hat auch uns fast jedes der dort monierten „Kleinigkeiten“ betroffen.Zusammengefasst ist dieses Etablissement nicht empfehlenswert und für den gebotenen Service zu teuer – Karlsruhe hat viele, deutlich bessere Lokale und unter gehobener Gastronomie stelle ich mir etwas anderes vor.",5,de,157,279


In [34]:
with open("../prompt_template.txt", "r", encoding="utf-8") as file:
    prompt_template = file.read()

In [35]:
template_size = num_tokens_from_string(prompt_template, LLM_MODEL_NAME)
aspect_term_tag_size = num_tokens_from_string('<aspect-term aspect="ambient/interior" polarity="negative"></aspect-term>', LLM_MODEL_NAME)

In [36]:
template_size + 10 * max_token_row["llm_token_count"].values[0] + aspect_term_tag_size * 20

3393